In [1]:
import pandas as pd

In [4]:

# === 1. Đọc lại các bảng đã tạo ===
fact_sales = pd.read_csv("Data/star_schema/fact_sales.csv")
dim_store = pd.read_csv("Data/star_schema/dim_store.csv")
dim_item = pd.read_csv("Data/star_schema/dim_item.csv")
dim_calendar = pd.read_csv("Data/star_schema/dim_calendar.csv")

# === 2. STORE-LEVEL AGGREGATION ===
store_level = (
    fact_sales.groupby(["store_id", "week_id"], as_index=False)
    .agg({
        "weekly_units_sold": "sum",
        "revenue": "sum"
    })
    .merge(dim_store, on="store_id", how="left")
)
store_level.rename(columns={
    "weekly_units_sold": "total_units_sold",
    "revenue": "total_revenue"
}, inplace=True)

# === 3. ITEM-LEVEL AGGREGATION ===
item_level = (
    fact_sales.groupby(["item_id", "week_id"], as_index=False)
    .agg({
        "weekly_units_sold": "sum",
        "revenue": "sum"
    })
    .merge(dim_item, on="item_id", how="left")
)
item_level.rename(columns={
    "weekly_units_sold": "total_units_sold",
    "revenue": "total_revenue"
}, inplace=True)

# === 4. DAILY-LEVEL (thực chất là WEEKLY-LEVEL) AGGREGATION ===
daily_level = (
    fact_sales.groupby(["week_id"], as_index=False)
    .agg({
        "weekly_units_sold": "sum",
        "revenue": "sum"
    })
    .merge(dim_calendar[["week_id", "year", "week_of_year", "event_name", "event_type"]],
           on="week_id", how="left")
)
daily_level.rename(columns={
    "weekly_units_sold": "total_units_sold",
    "revenue": "total_revenue"
}, inplace=True)

# === 5. Lưu kết quả ra CSV ===
store_level.to_csv("agg_store_level.csv", index=False)
item_level.to_csv("agg_item_level.csv", index=False)
daily_level.to_csv("agg_daily_level.csv", index=False)

print("✅ Đã tạo 3 bảng tổng hợp: store-level, item-level, daily-level!")

✅ Đã tạo 3 bảng tổng hợp: store-level, item-level, daily-level!
